<a href="https://colab.research.google.com/github/sachin886x/deep-learning-lab/blob/main/lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
# Load dataset
data = pd.read_csv("poems-100 - poems-100.csv")

text = " ".join(data["text"].astype(str)).lower()
words = text.split()

# Create vocabulary
vocab = sorted(set(words))
word_to_idx = {w:i for i,w in enumerate(vocab)}
idx_to_word = {i:w for w,i in word_to_idx.items()}
vocab_size = len(vocab)

# Create sequences
seq_length = 5
X = []
y = []

for i in range(len(words) - seq_length):
    X.append([word_to_idx[w] for w in words[i:i+seq_length]])
    y.append(word_to_idx[words[i+seq_length]])

X = torch.tensor(X)
y = torch.tensor(y)

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=16, shuffle=True)   # small batch size


In [ ]:
class OneHotRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(vocab_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = torch.nn.functional.one_hot(x, num_classes=vocab_size).float()
        x = x.to(device)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out


In [ ]:
class EmbeddingRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = x.to(device)
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out


In [ ]:
def train_model(model, epochs=10):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.003)

    for epoch in range(epochs):
        total_loss = 0

        for inputs, targets in loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(loader):.4f}")


In [ ]:
print("Training One-Hot RNN")
onehot_model = OneHotRNN(vocab_size, hidden_size=128)
train_model(onehot_model, epochs=20)

print("\nTraining Embedding RNN")
embedding_model = EmbeddingRNN(vocab_size, embed_dim=128, hidden_size=128)
train_model(embedding_model, epochs=20)


Training One-Hot RNN
Epoch 1/20, Loss: 7.3660
Epoch 2/20, Loss: 6.1605
Epoch 3/20, Loss: 4.8593
Epoch 4/20, Loss: 3.2055
Epoch 5/20, Loss: 1.6560
Epoch 6/20, Loss: 0.7567
Epoch 7/20, Loss: 0.4564
Epoch 8/20, Loss: 0.3461
Epoch 9/20, Loss: 0.2810
Epoch 10/20, Loss: 0.2434
Epoch 11/20, Loss: 0.2291
Epoch 12/20, Loss: 0.2112
Epoch 13/20, Loss: 0.2114
Epoch 14/20, Loss: 0.1841
Epoch 15/20, Loss: 0.1874
Epoch 16/20, Loss: 0.1836
Epoch 17/20, Loss: 0.1662
Epoch 18/20, Loss: 0.1841
Epoch 19/20, Loss: 0.1565
Epoch 20/20, Loss: 0.1688

Training Embedding RNN
Epoch 1/20, Loss: 7.5379
Epoch 2/20, Loss: 6.2877
Epoch 3/20, Loss: 4.9458
Epoch 4/20, Loss: 3.8443
Epoch 5/20, Loss: 3.0660
Epoch 6/20, Loss: 2.5496
Epoch 7/20, Loss: 2.1802
Epoch 8/20, Loss: 1.9390
Epoch 9/20, Loss: 1.7676
Epoch 10/20, Loss: 1.6537
Epoch 11/20, Loss: 1.5538
Epoch 12/20, Loss: 1.4731
Epoch 13/20, Loss: 1.4323
Epoch 14/20, Loss: 1.4110
Epoch 15/20, Loss: 1.3584
Epoch 16/20, Loss: 1.3473
Epoch 17/20, Loss: 1.3483
Epoch 18/20

In [ ]:
def generate_text(model, start_word, length=20):
    model.eval()
    result = [start_word]
    current = torch.tensor([[word_to_idx[start_word]]]).to(device)

    for _ in range(length):
        output = model(current)
        prob = torch.softmax(output, dim=1)
        next_idx = torch.multinomial(prob, 1).item()

        result.append(idx_to_word[next_idx])
        current = torch.tensor([[next_idx]]).to(device)

    return " ".join(result)


In [ ]:
print(generate_text(embedding_model, "love"))


love thee to be of the work and the work but i tell have learnt men and i know who would
